# Convolutional NN MNIST

In [16]:
from keras import layers
from keras import models
from keras.datasets import mnist # dataset
from keras.utils import to_categorical

In [13]:
model = models.Sequential()

# input shape: image height, width, depth (RGB)
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu')) 
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))


In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


Note that the above is just the definition of the general NN architecture, it has no classifier attached to it yet

In [15]:
model.add(layers.Flatten()) # flattening layers
model.add(layers.Dense(64, activation='relu')) # fitting a dense layer between the convnet and the output layer
model.add(layers.Dense(10, activation='softmax')) # 10 nodes because we have a 10-way classification

In [42]:
# compiling the model
model.compile(
    optimizer='rmsprop', # adam is possibly a better choice?
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [19]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data() # the dataset is lazily evaluated (need to download it)

11436032/11490434 [============================>.] - ETA: 0s  

The image data (`train_image`) is an array / tensor while the label data is the numeric vector corresponding to the number the image represents.

In [40]:
nn_data = {} # save results in a dictionary
for s in ['train','test']:
    target_images = locals()['{}_images'.format(s)] # pull relevant input data
    new_shape = target_images.shape + (1,)
    target_images = (
        target_images
        .reshape(new_shape) # pad the end with a one
        .astype('float32')
    ) / 255
    target_response = to_categorical(locals()['{}_labels'.format(s)]) # convert numbers to 0-1 matrix
    nn_data[s] = { # save in a dictionary
        'in': target_images,
        'out': target_response
    }

In [ ]:
# fitting (note: the machine may struggle with this one, so it can freeze at some point)
model.fit(nn_data['train']['in'], nn_data['train']['out'], epochs=5, batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 28s - loss: 0.1730 - acc: 0.9452      

In [45]:
test_loss, test_acc = model.evaluate(nn_data['test']['in'], nn_data['test']['out'])
print('\n')
print('test accuracy is {:.2%}'.format(test_acc))

10000/10000 [==============================] - 1s     